In [25]:
import pyfits, os, pylab
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Read folder name start with kplr
os.chdir("D:\starspiritstorm\Course\Meeting\Kepler\Kepler Revised Teff between 3800 to 3900\LC") # Change Directory
#nowpath = os.getcwd()

Parameter_file = 'Kepler K dwarf list Teff less than 3900 Quarter_counted.csv'
Parameter_data = pd.read_csv(Parameter_file)
df = pd.DataFrame(Parameter_data)

#make the folder list
Folder_list = 'Folder_list.txt'
Folder_name = file(Folder_list).readlines()

for i in Folder_name:
    path = i.replace('\n','')
    os.chdir("D:\starspiritstorm\Course\Meeting\Kepler\Kepler Revised Teff between 3800 to 3900\LC\%s"%path)
    
    #1 Read filename .fits
    nowpath = os.getcwd()
    filenamelist = filter(lambda x: x.endswith('.fits'), os.listdir(nowpath))
    
    #1.5 Write Raw data file into txt list (.fits)
    writelist = [ i +'\n' for i in filenamelist]
    with open('%s_Fits_list.txt' %path,'w') as f:
        map( f.write , writelist)
        
    #2 Read needed file data
    Kepler = '%s_Fits_list.txt' %path
    LC_file = file(Kepler).readlines()
    
    for j in LC_file:
        data=pyfits.open(j.replace('\n',''))
        time=data[1].data.field('TIME')
        sap_flux=data[1].data.field('SAP_FLUX')
        sap_flux_err=data[1].data.field('SAP_FLUX_ERR')
        pdcsap_flux=data[1].data.field('PDCSAP_FLUX')
        pdcsap_flux_err=data[1].data.field('PDCSAP_FLUX_ERR')
        lc=zip(time,sap_flux,sap_flux_err,pdcsap_flux,pdcsap_flux_err)
    
        output_file = j.replace('\n','') + '.lc'
    
    #2.5 Write needed data into a new file .lc
        out = open(output_file,'w')
        for i in lc:
            write = '%s %s %s %s %s' %(i[0],i[1],i[2],i[3],i[4])
            out.write(write+'\n')
        out.close()
    
    #3 Read picked up data in .lc and Write in txt list (.lc) 
    filenamelist = filter(lambda x: x.endswith('.lc'), os.listdir(nowpath))
    writelist = [ i +'\n' for i in filenamelist]
    with open('%s_list.txt' %path,'w') as f:
        map( f.write , writelist)
    
    #3.5  Read txt list (.lc) by lines
    Kepler = '%s_list.txt' %path
    LC_file = file(Kepler).readlines()
    
    MaxFlare_record = 0
    #4 Plot file in list
    for j in LC_file:
        filej = open(j.replace('\n',''))
        data = np.loadtxt( filej , dtype={'names':('time', '2', '3','flux','5'), 'formats':('f4','f4','f4','f4','f4')})
        masknan = -np.isnan(data['flux'])
        time = data['time'][masknan]
        normalized_flux = (data['flux'][masknan]-np.mean(data['flux'][masknan]))/np.mean(data['flux'][masknan])
        plt.plot(time, normalized_flux)
        plt.scatter(time, normalized_flux, marker='.')
        plt.xlabel('Julian Day - 2454833(day)', fontsize=12)
        plt.ylabel("$\mathrm{\mathsf{\Delta F/ F}}$", fontsize=12)
        plt.title('($\mathrm{\mathsf{\Delta F/ F}}$)max = %s'%np.max(normalized_flux))
        pylab.savefig('%s.png'%j.replace('.fits.lc\n','_maxflux=%s'%str(np.max(normalized_flux))))
        #plt.show()
        plt.clf()

        check_KID = ''
        if j[5] == '0':
            check_KID = j[6:13]
        if j[5] != '0':
            check_KID = j[5:13]

        if MaxFlare_record < np.max(normalized_flux):
            MaxFlare_record = np.max(normalized_flux)
        
        for j in range(len(Parameter_data['KIC'])):
            if str(Parameter_data['KIC'][j]) == str(check_KID):
                Parameter_data['Max Flareflux'][j] = MaxFlare_record


os.chdir("D:\starspiritstorm\Course\Meeting\Kepler\Kepler Revised Teff between 3800 to 3900\LC") # Change Directory
df.to_csv('Kepler K dwarf list Teff less than 3900 maximum flux recorded.csv')    

    


D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:68: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
